In [223]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [224]:
df = pd.DataFrame([], columns=['vacancy_id', 'company', 'description', 'schedule', 'city', 'salary'])

In [225]:
# df = df.append(dict(vacancy_id=1, company=2, description=3, schedule=4, city=5, salary=6), ignore_index=True)
# df.append(dict(vacancy_id=1, company=2, description=3, schedule=4, city=5, salary=6), ignore_index=True)

In [226]:
df

,vacancy_id,company,description,schedule,city,salary


In [227]:
job = 'клиент-менеджер'
location ='киев'

In [232]:
page_cnt = 101
vac_cnt = 2000
while True:
    print(f"Page #{page_cnt}")
    page = requests.get(f'https://rabota.ua/zapros/{job}/pg{page_cnt}')
    if page.status_code != 200 or page_cnt > 300:
        print(f"Finished on page {page_cnt} | # of vacancies: {vac_cnt}")
        break
    soup = BeautifulSoup(page.content)
    vacancies = re.findall('onclick="javascript: window.location = \'(.+)\'"', str(soup))
    for vacancy in vacancies:
        vacancy_data = requests.get(f"https://rabota.ua{vacancy}")
        if vacancy_data.status_code != 200:
            print(f"Finished on page {page_cnt} | # of vacancies: {vac_cnt}")
            break
        vacancy_soup = str(BeautifulSoup(vacancy_data.content))
        
        id = int(re.search('.+?([0-9]+).*', vacancy).groups()[0])
        salary = 0
        company = np.nan
        city = np.nan
        description = np.nan
        shedule = np.nan

        salary_match = re.search('"vacancy_Salary":([0-9]+),', vacancy_soup)
        if salary_match:
            salary = int(salary_match.groups()[0])
        city_match = re.search(r'"vacancy_CityName":"(.+?)"', vacancy_soup)
        if city_match:
            city = city_match.groups()[0]
        desc_match = re.search(r'"vacancy_Description":"(.+)","vacancy_NotebookId"', vacancy_soup)
        if desc_match:
            description = desc_match.groups()[0]
        company_match = re.search(r'"vacancyCompanyName":"(.+?)"', vacancy_soup)
        if company_match:
            company = company_match.groups()[0]
        shedule_match = re.search(r'"vacancy_ScheduleName":"(.+?)",', str(vacancy_soup))
        if shedule_match:
            shedule = shedule_match.groups()[0]
        df = df.append(dict(vacancy_id=id, company=company, description=description,
                            schedule=shedule, city=city, salary=salary), ignore_index=True)
        vac_cnt += 1
        if vac_cnt > 0 and vac_cnt % 100 == 0:
            print(f"[{vac_cnt}] vacancies, example vacancy:")
            print(df.tail(n=1))
    page_cnt += 1

Page #101
Page #102
Page #103
Page #104
Page #105
[2100] vacancies, example vacancy:
     vacancy_id           company  \
2099    5023044  Качура Е.А., ФЛП   

                                            description          schedule  \
2099  <p><b>Хочешь заработать много денег, получить ...  полная занятость   

      city salary  
2099  Киев  12000  
Page #106
Page #107
Page #108
Page #109
Page #110
[2200] vacancies, example vacancy:
     vacancy_id            company  \
2199    1034962  Центрум-ЛТД, ООО    

                                            description          schedule  \
2199  <p><b>Компания \"Центрум\" уже более 14 лет за...  полная занятость   

         city salary  
2199  Харьков  30000  
Page #111
Page #112
Page #113
Page #114
Page #115
[2300] vacancies, example vacancy:
     vacancy_id    company                                        description  \
2299     850430  Мамин Дом  <p><b>В связи с развитием, компания Мамин Дом ...   

              schedule     city sa

In [236]:
df.to_csv("scrapped_data.csv", index=False)